### Import the required libraries and read the file containing 2024 crime data.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.neighbors import KNeighborsRegressor
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn import metrics
from sklearn.ensemble import ExtraTreesRegressor

In [ ]:
crime_df = pd.read_excel('Regbcs 2009-2023_2024_I-IX.xlsx', sheet_name="2024_I-IX_hó" ,engine='openpyxl')

###Convert Dataset
Ensure that the 2024 crime data is aligned with the dataset used to train the model. This alignment is essential for validating the model’s performance and ensuring consistent predictions. Begin by determining the city category based on population, then remove cities with populations under 1,000. Next, drop any missing values and exclude foreign crime data. Categorize the cities and calculate the crime rate, adding 25% to account for a full year. Finally, remove the top and bottom 0.001% of the data to eliminate outliers.

In [ ]:
import pandas as pd

# Load the population data
population_data = pd.read_excel('Population.xlsx', engine='openpyxl')

# Normalize location names in both datasets (remove extra spaces, standardize formats if needed)
crime_df['Elkövetés helye'] = crime_df['Elkövetés helye'].str.strip()
population_data['Location'] = population_data['Location'].str.strip()

# Drop duplicates from population_data to ensure no duplicate location entries
population_data = population_data.drop_duplicates(subset='Location')

# Merge the datasets on the location names
merged_data = pd.merge(crime_df, population_data, left_on='Elkövetés helye', right_on='Location', how='left')

# Dropping rows where population is null or City is (üres) or population is below 1000
merged_data = merged_data[merged_data['Population'].notna() & (merged_data['Elkövetés helye'] != '(üres)') & (merged_data['Population'] >= 1000)]

# Assign population categories
def categorize_population(population):
    if 100000 <= population <= 500000:
        return 'Nagyváros'
    elif 10000 <= population < 100000:
        return 'Középváros'
    elif 5000 <= population < 10000:
        return 'Kisváros'
    elif 2000 <= population < 5000:
        return 'Nagyfalvak'
    else:
        return 'Középfalvak'

merged_data['City Category'] = merged_data['Population'].apply(categorize_population)

# Drop the redundant 'Location' column from the population data
merged_data = merged_data.drop(columns=['Location'])

# Define function to calculate crime rate as a percentage based on the city category
def calculate_crime_rate(row):
    population = row['Population']
    crimes = row['Regisztrált bűncselekmények száma']

    if row['City Category'] == 'Nagyváros':
        return round((crimes / (population / 100000)), 4)*1.25 # adjust the crime rate (add + 25%)
    elif row['City Category'] == 'Középváros':
        return round((crimes / (population / 1000)), 4)*1.25
    elif row['City Category'] == 'Kisváros':
        return round((crimes / (population / 1000)), 4)*1.25
    elif row['City Category'] == 'Nagyfalvak':
        return round((crimes / (population / 100)), 4)*1.25
    elif row['City Category'] == 'Középfalvak':
        return round((crimes / (population / 100)), 4)*1.25
    else:
        return None

# Apply the function to create the new 'Crime Rate' column
merged_data['Crime Rate'] = merged_data.apply(calculate_crime_rate, axis=1)

#Remove the top and bottom 0.001% of the data to eliminate outliers
min_treshold, max_treshold = merged_data['Crime Rate'].quantile([0.001,0.9990])
merged_data = merged_data[(merged_data['Crime Rate'] >= min_treshold) & (merged_data['Crime Rate'] <= max_treshold)]


# Save the merged DataFrame to an Excel file
merged_data.to_excel('2024preprocessed_crime_data.xlsx', index=False, engine='openpyxl')


<ipython-input-69-ab1013f23a12>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data['City Category'] = merged_data['Population'].apply(categorize_population)


### Preprocess 2024 crime data for model predictions.

Encode the variables in the 2024 crime dataset to match the format and structure of the original training dataset. Ensure that categorical variables are transformed using the same encoding methods, and numerical variables are scaled or normalized consistently. **(city, city_category, crime_type)**

In [70]:
import pandas as pd

# Load the main Excel file with columns 'City', 'Category', and 'Crime Type'
main_file = pd.read_excel('2024preprocessed_crime_data.xlsx')  # Replace with your filename
main_file = main_file.dropna()

# Helper function to read a TXT file and create a dictionary
def load_mapping(file_path):
    mapping = {}
    with open(file_path, 'r') as f:
        for line in f:
            key, value = line.strip().split(':')
            mapping[key.strip()] = int(value.strip())  # Convert value to int if needed
    return mapping

# Load all mappings from respective TXT files
city_mapping = load_mapping('City_Mapping.txt')
category_mapping = load_mapping('City_Category_Mapping.txt')
crime_type_mapping = load_mapping('Crime_Type_Mapping.txt')

# Apply the mappings to respective columns in the DataFrame
main_file['Elkövetés helye'] = main_file['Elkövetés helye'].map(city_mapping)
main_file['City Category'] = main_file['City Category'].map(category_mapping)
main_file['Bűncselemény'] = main_file['Bűncselemény'].map(crime_type_mapping)

# Save the input file (without dropping columns)
main_file.to_excel('test_output.xlsx', index=False)

# Drop the specified columns and save the output file
columns_to_drop = ['Regisztrált bűncselekmények száma', 'Crime Rate']  # Update column names as needed
main_file.drop(columns=columns_to_drop, inplace=True)
main_file.to_excel('test_input.xlsx', index=False)


### Train the original model.
Train the original model using the already preprocessed data.

In [71]:
real_df = pd.read_excel('merged_crime_population_final.xlsx' ,engine='openpyxl')

In [72]:
x = real_df[real_df.columns[0:5]].values
x

array([[   229,    140,   2019, 122973,      4],
       [   229,    140,   2022, 122973,      4],
       [   229,    398,   2019,  32625,      2],
       ...,
       [   226,   1282,   2011,   6510,      0],
       [   226,   1284,   2010,   1558,      1],
       [   226,   1285,   2016,   3373,      3]])

In [73]:
y = real_df['Crime Rate'].values
y

array([0.8132, 3.2527, 0.0307, ..., 0.4608, 0.0642, 0.0296])

In [74]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=48)

In [75]:
model2 = RandomForestRegressor(n_jobs=-1, random_state=574)
model2.fit(x_train, y_train)
y_pred = model2.predict(x_test)

In [76]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('R2 score:', metrics.r2_score(y_test, y_pred))


Mean Absolute Error: 0.502419632997934
Mean Squared Error: 13.772615175535513
R2 score: 0.8443686176579677


### Make Predictions on 2024 Crime Data.
Read the input file containing the independent variables for the 2024 crime data and use the original model to make predictions based on this data.

In [91]:
x_real_test = pd.read_excel('test_input.xlsx') # Input file with independent variables
real_test_predictions = model2.predict(x_real_test)

# Display the predictions
print("2024 data predictions:")
print(real_test_predictions)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


2024 data predictions:
[0.040413 0.022105 0.100141 ... 0.047824 0.073266 0.127405]


###Read the File Containing the actual 2024 Crime Rates.

Read the file, which contains the actual crime rates for the 2024 data.

In [81]:
test = pd.read_excel('test_output.xlsx')
y_real_test = test['Crime Rate'] # actual crimes rate for 2024
print(y_real_test)

0        0.029750
1        0.023000
2        0.076125
3        0.057000
4        0.067750
           ...   
18338    0.059125
18339    0.060250
18340    0.022375
18341    0.045875
18342    0.052125
Name: Crime Rate, Length: 18343, dtype: float64


### Evaluate the Model's performance using the 2024 Crime Data.





In [92]:
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error, r2_score
import numpy as np

mse = mean_squared_error(y_real_test, real_test_predictions)
mae = mean_absolute_error(y_real_test, real_test_predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_real_test, real_test_predictions)
rmsle = np.sqrt(mean_squared_log_error(y_real_test, real_test_predictions))

metrics_data = {
    "Data": ["2024 Crime Data"],
    "MAE": [mae],
    "MSE": [mse],
    "RMSE": [rmse],
    "R²": [r2],
    "RMSLE": [rmsle]
}

metrics_df = pd.DataFrame(metrics_data).round(4)

# Styling the DataFrame
def highlight_values(val):
    return "background-color: yellow;"

styled_df = metrics_df.style.applymap(highlight_values)
styled_df

<ipython-input-92-4507e6e4aa76>:26: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = metrics_df.style.applymap(highlight_values)


,Data,MAE,MSE,RMSE,R²,RMSLE
0,2024 Crime Data,0.440200,10.237800,3.199700,0.824500,0.224400
